In [ ]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras

In [ ]:
def loadCombinedArray(cases,field):
    data = np.concatenate([np.load('../data/'+dataset+'/'+dataset+'_'+case+'_'+field + '.npy') for case in cases])
    return data


In [ ]:
dataset = "kepsilon"
cases   = ["PHLL_case_1p0"]

In [ ]:
x  = loadCombinedArray(cases,"Cx")
y  = loadCombinedArray(cases,"Cy")
z  = loadCombinedArray(cases,"Cz")
Ux = loadCombinedArray(cases,"Ux")
Uy = loadCombinedArray(cases,"Uy")
Uz = loadCombinedArray(cases,"Uz")
p  = loadCombinedArray(cases,"p")


In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

ax.scatter(x,y,c="k",s=0.01)

In [ ]:
fig, ax = plt.subplots(4,figsize=(6,8),sharex=True)

ax[0].tricontourf(x,y,Ux,levels=50,cmap="jet")
ax[1].tricontourf(x,y,Uy,levels=50,cmap="jet")
ax[2].tricontourf(x,y,Uz,levels=50,cmap="jet")
ax[3].tricontourf(x,y,p, levels=50,cmap="jet")

In [ ]:
# get data ready in an array
# scale the data
# be clear what are the features (X) , and what are target (Y)
# X[Ux,Uy,Uz], Y[p]
data_combined = np.column_stack((Ux,Uy,Uz,p))
df = pd.DataFrame(data=data_combined)
df.columns=["Ux","Uy","Uz","p"]
print(df.describe())
df.hist(bins=30)

In [ ]:
df_features = df[["Ux","Uy"]]
df_target   = df["p"]

scaler_features = preprocessing.MinMaxScaler()
scaler_target   = preprocessing.MinMaxScaler()

df_features = pd.DataFrame(scaler_features.fit_transform(df_features),
                          columns=["Ux","Uy"])
df_target = pd.DataFrame(scaler_target.fit_transform(df_target.values.reshape(-1,1)),
                          columns=["p"])

In [ ]:
# df_features

In [ ]:
# Building my NN
keras.backend.clear_session()

#Ux, Uy
input_layer = keras.layers.Input(shape=(2),name="input_layer")

#Hidden layers
hidden1     = keras.layers.Dense(20,name="Hidden1",kernel_initializer="lecun_normal",
                                activation="selu")(input_layer)
hidden2 = keras.layers.Dense(20,name='Hidden2', kernel_initializer="lecun_normal", activation = "selu")(hidden1)
hidden3 = keras.layers.Dense(20,name='Hidden3', kernel_initializer="lecun_normal", activation = "selu")(hidden2)
hidden4 = keras.layers.Dense(20,name='Hidden4', kernel_initializer="lecun_normal", activation = "selu")(hidden3)
hidden5 = keras.layers.Dense(20,name='Hidden5', kernel_initializer="lecun_normal", activation = "selu")(hidden4)


output_layer = keras.layers.Dense(1,name="output_layer")(hidden5)

model=keras.Model(inputs=[input_layer], outputs=[output_layer])
model.summary()


In [ ]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(optimizer,loss="mse",metrics=["mae","mse"])

In [ ]:
history = model.fit([df_features],df_target,
                    epochs=200,
                    verbose= 0,
)

In [ ]:
predictions = model.predict(df_features).reshape(len(df_features))
truth = df_target['p']

fig = plt.figure(figsize=(12,12))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ax1.tricontourf(x,y,predictions,levels=50,vmin=0,vmax=1)
ax1.set_aspect(1)
ax1.set_title('Predictions')
ax2.tricontourf(x,y,truth,levels=50,vmin=0,vmax=1)
ax2.set_aspect(1)
ax2.set_title('Truth')